## Serves as a advanced node model which focuses on building model on multiple nodes

### import packages and read data

In [3]:
!pip install /root/git/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
from msspackages import setup_runner
setup_runner(setup_type = 'notebook' , project = 'understanding-eks-data')

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/git/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
msspackages is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
b"Get:1 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]\nHit:2 http://deb.debian.org/debian buster InRelease\nHit:3 http://deb.debian.org/debian buster-updates InRelease\nGet:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [412 kB]\nFetched 447 kB in 0s (1528 kB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\n52 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nsudo is already the newest version (1.8.27-1+deb10u4).\n0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.\nReading package lists...\nBuilding dep

In [4]:
%pip install hmmlearn
from msspackages import Pyspark_data_ingestion
from pyspark import StorageLevel
from pyspark.sql.functions import *
from hmmlearn import hmm,base
from sklearn.preprocessing import StandardScaler as scale
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np
# pandas settings 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.


In [7]:
%%time
node_obj = Pyspark_data_ingestion(year = '2022', month = "7", day = "7", hour = 10, filter_column_value ='Node',setup='128gb')
spark_node = node_obj.get_spark()
err, node_data = node_obj.read()

CPU times: user 31.1 ms, sys: 11.7 ms, total: 42.9 ms
Wall time: 9.09 s


In [9]:
df = node_data.toPandas()

In [10]:
import warnings
warnings.filterwarnings('ignore')


In [11]:
df.to_csv('one_hour.csv', index=False, header=True)

In [ ]:
df.shape

In [ ]:
def clean_data(df):
    df = df.dropna(subset=['node_cpu_utilization','node_memory_utilization'])
    nodelist = df.NodeName.unique()
    subdfs = []
    for node in nodelist:
        
        subdf = df[df.NodeName==node]
        sc_m = scale()
        sc_c = scale()
        c_s = sc_c.fit_transform(subdf['node_cpu_utilization'].values.reshape(-1,1))
        m_s = sc_m.fit_transform(subdf['node_memory_utilization'].values.reshape(-1,1))
        subdf['node_cpu_utilization'] = c_s
        subdf['node_memory_utilization'] = m_s
        subdfs.append(subdf)
    cleaned = subdfs[0]
    
    for i in subdfs[1:]:
        cleaned = pd.concat([cleaned, i], axis=0)
    return cleaned

## feature selection

In [ ]:
len(new_df)

In [ ]:
fig,ax = plt.subplots(len(new_df.NodeName.unique()),figsize = (10,40),sharex=True)
i = 0
for node in new_df.NodeName.unique():
    sub = new_df[new_df.NodeName==node]
    ax[i].hist(sub.node_cpu_utilization)
    i+=1
plt.show()
    

In [ ]:
new_df.reset_index(inplace=True,drop=True)

In [ ]:
new_df.head()

In [ ]:
def sample_data(df,data_frac= 0.5,random_state = 1, slice_length = 6):
    res_ind = df.sample(frac=data_frac/6, replace=True, random_state=random_state).index.to_list()
    for i in res_ind :
        for j in range(slice_length):
            res_ind.append(i+j)
            
    return df.iloc[res_ind.sort()]
        
        
    

In [ ]:
res_ind = new_df.sample(frac=0.5/6, replace=True, random_state=1).index.to_list()

In [ ]:

for i in res_ind :
    for j in range(6):
        res_ind.append(i+j)

In [ ]:


res_ind[:5]

In [ ]:
new_df.iloc[[5,5]]

## model testing V1.0

### preprocess

In [10]:
features = ['node_cpu_utilization','node_memory_utilization']
#filter inital node df based on request features
node_df =df.select("Timestamp", "NodeName", 'node_cpu_utilization','node_memory_utilization')
node_df = node_df.withColumn("Timestamp",(col("Timestamp")/1000).cast("timestamp"))

# Drop NA
cleaned_node_df = node_df.na.drop(subset=features)


#Quality(timestamp filtered) nodes
quality_filtered_node_df = cleaned_node_df.groupBy("NodeName").agg(count("Timestamp").alias("timestamp_count"))
quality_filtered_nodes = quality_filtered_node_df.filter(col("timestamp_count").between(45,75))

print(3)
#Processed Node DF                                                      
processed_node_df = cleaned_node_df.filter(col("NodeName").isin(quality_filtered_nodes["NodeName"]))



AttributeError: 'DataFrame' object has no attribute 'select'

### split data

In [ ]:
input_df, test_df = processed_node_df .randomSplit(weights=[0.8,0.2], seed=200)

In [ ]:
input_df.show(n=1)

### feature engineering

In [ ]:
n = 0
samplesize =1
time_steps=12
final_df = np.zeros((samplesize,time_steps,len(features)+1))

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

In [ ]:
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.ml import Pipeline


In [ ]:
import random
##pick random node
random_nodename = random.choice(input_df.select("NodeName").rdd.flatMap(list).collect())
node_df = input_df[(input_df["NodeName"] ==  random_nodename)][["Timestamp", "NodeName"] + features].select('*')
node_df = node_df.sort("Timestamp")
node_df = node_df.na.drop(subset=features)
    


#standardize data from the node


w = Window.partitionBy('NodeName')
for c in features:
    node_df = (node_df.withColumn('mean', F.mean(c).over(w))
        .withColumn('stddev', F.stddev(c).over(w))
        .withColumn(c, ((F.col(c) - F.col('mean')) / (F.col('stddev'))))
        .drop('mean')
        .drop('stddev'))


    

In [ ]:
pwd

In [ ]:
node_df.columns

In [ ]:
#pick random time slice of 12 timestamps from this node
final_df = None
start = random.choice(range(node_df.count()-time_steps))
node_slice_df = node_df.withColumn('rn', row_number().over(Window.orderBy("Timestamp"))).filter((col("rn") >= start) & (col("rn") < start+time_steps)).select(["Timestamp"] + features)
node_uti_data = node_slice_df.select(*features).rdd.flatMap(list).collect()

if not final_df:
    final_df = node_slice_df 
else:
    final_df = final_df.union(node_slice_df )


In [ ]:
final_df = final_df.groupBy("Timestamp").mean()

In [ ]:
final_df = final_df.withColumnRenamed("avg(node_cpu_utilization)","node_cpu_utilization") \
    .withColumnRenamed("avg(node_memory_utilization)","node_memory_utilization")
    
final_df = final_df.select(*features)
    
   

In [ ]:
final_df.show(n=5,vertical=True)
        

In [ ]:
fianl_df

## Data cleaning

In [ ]:
df.head()

## model V2.0

In [4]:
df = pd.read_csv('one_hour.csv')

In [12]:
df.head()

,account_id,log_group_name,log_stream_name,record_id,stream_name,record_arrival_stream_timestamp,record_arrival_stream_epochtime,log_event_timestamp,log_event_epochtime,log_event_id,AutoScalingGroupName,CloudWatchMetrics,ClusterName,InstanceId,InstanceType,NodeName,Sources,Timestamp,Type,Version,kubernetes,node_cpu_limit,node_cpu_request,node_cpu_reserved_capacity,node_cpu_usage_system,node_cpu_usage_total,node_cpu_usage_user,node_cpu_utilization,node_memory_cache,node_memory_failcnt,node_memory_hierarchical_pgfault,node_memory_hierarchical_pgmajfault,node_memory_limit,node_memory_mapped_file,node_memory_max_usage,node_memory_pgfault,node_memory_pgmajfault,node_memory_request,node_memory_reserved_capacity,node_memory_rss,node_memory_swap,node_memory_usage,node_memory_utilization,node_memory_working_set,node_network_rx_bytes,node_network_rx_dropped,node_network_rx_errors,node_network_rx_packets,node_network_total_bytes,node_network_tx_bytes,node_network_tx_dropped,node_network_tx_errors,node_network_tx_packets,node_number_of_running_containers,node_number_of_running_pods,region
0,346687249423,/aws/containerinsights/nk-ndc-eks-cluster-test...,ip-100-64-19-132.us-west-2.compute.internal,4962796749719051212894665848731042236882148557...,dp-us-west-2-container-insights-performance-me...,2022-07-07 10:06:23,1657188383,2022-07-07 10:06:17,1657188377000,3695653574144350615072363607584040779097551619...,nk-ndc-asbc-asg-cicdtest-usw2-az2-perf-0,"[{""Dimensions"":[[""NodeName"",""InstanceId"",""Clus...",nk-ndc-eks-cluster-test-dev-usw2-az2-perf,i-0fd528a5b4db63a58,m5.16xlarge,ip-100-64-19-132.us-west-2.compute.internal,"[""cadvisor"",""/proc"",""pod"",""calculated""]",1657188368827,Node,0,"{""host"":""ip-100-64-19-132.us-west-2.compute.in...",64000.0,2716.0,4.243750,405.150452,684.565359,388.077959,1.069633,5686312960,0,0.0,0.0,267384934400,804790272,21409251328,0.0,0.0,3340763136,1.249421,2185527296,0,7871840256,1.789627,4785192960,33970.681319,0.0,0.0,138.360623,6.794702e+04,33976.335434,0.0,0.0,136.763519,18,12,us-west-2
1,866535947663,/aws/containerinsights/mt-ndc-eks-cluster-dev-...,ip-172-24-67-95.ec2.internal,4962796749719051212894665848741318106348872912...,dp-us-west-2-container-insights-performance-me...,2022-07-07 10:06:24,1657188384,2022-07-07 10:06:19,1657188379000,3695653578604499654778488236066210175310495798...,mt-ndc-chf-asg-dev-ASG01-non-multus,"[{""Dimensions"":[[""NodeName"",""InstanceId"",""Clus...",mt-ndc-eks-cluster-dev-dev-e1-cluster,i-0f83dfb0d9ea9120a,m5.4xlarge,ip-172-24-67-95.ec2.internal,"[""cadvisor"",""/proc"",""pod"",""calculated""]",1657188370769,Node,0,"{""host"":""ip-172-24-67-95.ec2.internal""}",16000.0,1681.0,10.506250,27.218900,82.893135,51.012905,0.518082,4468023296,0,0.0,0.0,65899286528,585142272,5688471552,0.0,0.0,1166016512,1.769392,546852864,0,5014876160,2.020823,1331707904,8701.016984,0.0,0.0,38.809464,2.256991e+04,13868.894612,0.0,0.0,38.070408,15,11,us-west-2
2,294485975972,/aws/containerinsights/mv-ndc-eks-cluster-dev-...,ip-172-16-50-125.us-west-2.compute.internal,4962796749719051212894665848743252387660256319...,dp-us-west-2-container-insights-performance-me...,2022-07-07 10:06:25,1657188385,2022-07-07 10:06:20,1657188380000,3695653580834574174631550550298854738425925581...,mv-ndc-scyla-asg-dev-usw2b-01-ASGusw2b-01,"[{""Dimensions"":[[""NodeName"",""InstanceId"",""Clus...",mv-ndc-eks-cluster-dev-vmas-usw2-01,i-0294f49d42a8a6084,m5n.16xlarge,ip-172-16-50-125.us-west-2.compute.internal,"[""cadvisor"",""/proc"",""pod"",""calculated""]",1657188378585,Node,0,"{""host"":""ip-172-16-50-125.us-west-2.compute.in...",64000.0,17266.0,26.978125,79.040780,276.004781,163.460517,0.431257,5378805760,0,0.0,0.0,267384934400,850206720,20248190976,0.0,0.0,69675778048,26.058229,5288857600,0,10667663360,2.965135,7928324096,29761.632839,0.0,0.0,77.710982,7.354837e+04,43786.739429,0.0,0.0,71.868838,15,11,us-west-2
3,237450111201,/aws/containerinsights/oracle-ndc-eks-cluster-...,ip-172-16-69-112.us-west-2.compute.

In [34]:
import datetime

df = df[['Timestamp','InstanceId','node_cpu_utilization','node_memory_utilization']]
df = df.dropna(subset=['node_cpu_utilization','node_memory_utilization'])
df['Datetime'] = df['Timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)/1000))
ls = df.groupby([df.InstanceId]).count().reset_index()
instancelist = ls[(ls.Datetime>=45)&(ls.Datetime<=75)].InstanceId.to_list()


In [36]:
%%time
df['split'] = df['InstanceId'].apply(lambda x:x in instancelist)

CPU times: user 298 ms, sys: 0 ns, total: 298 ms
Wall time: 296 ms


In [38]:
df = df[df['split']==1]

## Data split

In [5]:
df = pd.read_csv('preprocessed.csv')

In [6]:
df.head()

,Unnamed: 0,Timestamp,InstanceId,scaled_mem,scaled_cpu
0,0,1657188368827,i-0fd528a5b4db63a58,0.929672,1.821592
1,1,1657188370769,i-0f83dfb0d9ea9120a,0.306372,-0.288206
2,2,1657188378585,i-0294f49d42a8a6084,1.929476,0.305287
3,3,1657188380656,i-0135cde94606361ca,1.325316,3.473258
4,4,1657188381211,i-052ff6e758e5066a8,0.059157,-0.148335


In [11]:
## pick random nodes (half of the total nodes) and add them to our test data
import random
instancelist = df['InstanceId'].unique()
testlist = random.choices(instancelist, k=int(len(instancelist)/2))

In [13]:
df['x_train'] = df['InstanceId'].apply(lambda x:x in testlist)
x_train = df[df['x_train']==1]
x_test= df[df['x_train']==0]

,Unnamed: 0,Timestamp,InstanceId,scaled_mem,scaled_cpu,x_train
2,2,1657188378585,i-0294f49d42a8a6084,1.929476,0.305287,True
3,3,1657188380656,i-0135cde94606361ca,1.325316,3.473258,True
4,4,1657188381211,i-052ff6e758e5066a8,0.059157,-0.148335,True
7,7,1657188380255,i-019763c52c2720665,0.292811,1.425888,True
8,8,1657188381686,i-0c637059400cc0a16,0.002217,0.230518,True


In [15]:
x_train.to_csv('train.csv')
x_test.to_csv('test.csv')

## Feature Engineering

In [68]:
features = ['node_cpu_utilization','node_memory_utilization']
params_miu_cpu = df.groupby(by='InstanceId').mean()[features[0]]
params_miu_mem = df.groupby(by='InstanceId').mean()[features[1]]
params_sigma_cpu = df.groupby(by='InstanceId').std()[features[0]]
params_sigma_mem = df.groupby(by='InstanceId').std()[features[1]]
    

In [69]:

params_miu_cpu = params_miu_cpu.to_dict()
params_miu_mem = params_miu_mem.to_dict()
params_sigma_cpu = params_sigma_cpu.to_dict()
params_sigma_mem = params_sigma_mem.to_dict()

df['miu_cpu'] = df['InstanceId'].map(params_miu_cpu)
df['miu_mem'] = df['InstanceId'].map(params_miu_mem)
df['sigma_cpu'] = df['InstanceId'].map(params_sigma_cpu)
df['sigma_mem'] = df['InstanceId'].map(params_sigma_mem)


In [71]:
df['scaled_mem']=(df['node_memory_utilization']-df['miu_mem'])/df['sigma_mem']
df['scaled_cpu']=(df['node_cpu_utilization']-df['miu_cpu'])/df['sigma_cpu']

In [75]:
df.to_csv('preprocessed.csv')

In [ ]:
'''
import random
n = 0
features = ['node_cpu_utilization','node_memory_utilization']
instancelist = node_train.InstanceId.unique()
result = None
while n<samplesize:
    instance = random.choice(instancelist)
    subdf = df[df.InstanceId==instance].sort_values(by = ['Timestamp'])
    if len(subdf)<12:
        continue
    start = random.choice(range(len(subdf)-12))
    timeslice = subdf[start:start+12][['Timestamp',*features]]
    if n!=0:
        result = pd.concat([result,timeslice])
       
    else:
        result =  timeslice
    n+=1
    if n%100==0:
        print(n)
#result = result.groupby(by=['Timestamp']).mean()
'''

In [ ]:
result.to_csv('/root/git/eks-ml-pipeline/notebooks/train.csv', index=False, header=False)


## Train my model

In [16]:
x_train.head()


,Unnamed: 0,Timestamp,InstanceId,scaled_mem,scaled_cpu,x_train
2,2,1657188378585,i-0294f49d42a8a6084,1.929476,0.305287,True
3,3,1657188380656,i-0135cde94606361ca,1.325316,3.473258,True
4,4,1657188381211,i-052ff6e758e5066a8,0.059157,-0.148335,True
7,7,1657188380255,i-019763c52c2720665,0.292811,1.425888,True
8,8,1657188381686,i-0c637059400cc0a16,0.002217,0.230518,True


In [17]:
x_train = x_train[['Timestamp','InstanceId','scaled_mem','scaled_cpu']]
#generate test sequences
x_train = x_train.sort_values(by=['InstanceId','Timestamp'])

In [19]:
test_data = []
for i in testlist:
    subdf = x_train[x_train['InstanceId']==i]
    test_data.append(subdf[['scaled_cpu','scaled_mem']].values)
    

In [53]:

model = hmm.GaussianHMM(n_components=2, covariance_type="full",init_params='')
model.startprob_prior = 2.0
model.fit(test_data[0])

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
            covars_weight=1, implementation='log', init_params='',
            means_prior=0, means_weight=0, min_covar=0.001, n_components=2,
            n_iter=10, params='stmc', random_state=None, startprob_prior=2.0,
            tol=0.01, transmat_prior=1.0, verbose=False)

In [57]:
##test
model = hmm.GaussianHMM(n_components=2, covariance_type="full",init_params='')
model.startprob_prior

    
    


1.0

In [68]:
t_df = test_data[3]
model.startprob_prior

1.0

In [95]:
## init mpdel
model = hmm.GaussianHMM(n_components=4, covariance_type="full",init_params='')
for i in range(len(test_data)):
    data = test_data[i]   
    model.fit(data)
 

    model.startprob_prior = model.startprob_
    model.transmat_prior = model.transmat_
    model.means_prior = model.means_
    model.covars_prior = model.covars_
    
    print(f"{i},\n startprob is \n {model.startprob_},\n transmat is \n{ model.transmat_}")
    

0,
 startprob is 
 [1.58396230e-13 7.41542916e-03 9.92584571e-01 5.28526598e-28],
 transmat is 
[[2.20320839e-03 2.92942174e-06 1.37951394e-01 8.59842468e-01]
 [1.42209700e-04 2.36214805e-01 7.63613251e-01 2.97346477e-05]
 [1.60129878e-02 8.29282687e-01 2.71947208e-02 1.27509605e-01]
 [9.74000174e-01 6.29770541e-04 1.81862735e-04 2.51881931e-02]]
1,
 startprob is 
 [0. 0. 1. 0.],
 transmat is 
[[0.         0.         0.28626106 0.71373894]
 [0.         0.44820913 0.55179087 0.        ]
 [0.         0.93014615 0.         0.06985385]
 [0.70669673 0.         0.         0.29330327]]
2,
 startprob is 
 [0. 0. 1. 0.],
 transmat is 
[[0.         0.         0.23566812 0.76433188]
 [0.         0.02701138 0.97298862 0.        ]
 [0.         0.87620107 0.         0.12379893]
 [0.87331287 0.         0.         0.12668713]]


/opt/conda/lib/python3.7/site-packages/hmmlearn/hmm.py:275: RuntimeWarning: invalid value encountered in true_divide
  / (means_weight + denom))


ValueError: array must not contain infs or NaNs

## Model 3.0
#### Will use the previous model before I figure out how to deal with singularity...